In [1]:
import sys
sys.path.append('../')
import timm
from stochastic_attention import StochasticVisionTransformer
from model_factory import create_model

/home/clement/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
timm_model = timm.create_model('vit_base_patch32_384')
model = StochasticVisionTransformer(embed_dim=768, depth=12, num_heads=12, kernel_size=32, qkv_bias=True)

Image size 512, stride 16, kernel size 32, out_chans 768, output size 31


In [3]:
my_state_dict = model.state_dict()
timm_state_dict = timm_model.state_dict()

In [4]:
blocks_depth = model.blocks_depth
for i, block in enumerate(timm_model.blocks):
    block_scale, block_level = divmod(i, blocks_depth)
    keys = model.blocks[block_scale].blocks[block_level].load_state_dict(block.state_dict())

In [5]:
model.projector.load_state_dict(timm_model.patch_embed.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['pos_embed'], unexpected_keys=[])

In [16]:
import torch
model = create_model('svt_16_small', num_classes=1000, img_size=512, max_tokens=256, pretrained=False, scales=3, projection_stride=4)
foo = torch.randn(2, 3, 512, 512)
pred, attn = model(foo)

Image size 512, stride 4, kernel size 16, out_chans 384, output size 125


AssertionError: 

torch.Size([1000])